In [1]:
import gc
import pandas as pd
import numpy as np
import datetime as dt
from time import time

# pleaseee uncomment the lines below if it is not installed on your local
# %pip install matplotlib
# %pip install seaborn
# %pip install plotly
# %pip install polars
# %pip install pyarrow

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import pyarrow

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.offline

from colorama import Fore, Style, init
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

In [2]:
%%time
# inspired by https://www.kaggle.com/code/enricomanosperti/detect-sleep-states-first-preprocessing-and-eda
import polars as pl
train_series_multi = (pl.scan_parquet('../data/Zzzs_train_multi.parquet')
                .with_columns(
                    (
                        (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")),
                        (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z").dt.year().alias("year")),
                        (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z").dt.month().alias("month")),
                        (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z").dt.day().alias("day")),
                        (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z").dt.hour().alias("hour")),
                    )
                )
                .collect()
                .to_pandas()
               )

CPU times: total: 12.9 s
Wall time: 21.6 s


In [3]:
%%time
# inspired by https://www.kaggle.com/code/enricomanosperti/detect-sleep-states-first-preprocessing-and-eda
import polars as pl
test_series = (pl.scan_parquet('../data/test_series.parquet')
                .with_columns(
                    (
                        (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")),
                        (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z").dt.year().alias("year")),
                        (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z").dt.month().alias("month")),
                        (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z").dt.day().alias("day")),
                        (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z").dt.hour().alias("hour")),
                    )
                )
                .collect()
                .to_pandas()
               )

CPU times: total: 15.6 ms
Wall time: 7.22 ms


In [4]:
train_series_multi.head()

,series_id,step,timestamp,anglez,enmo,awake,year,month,day,hour
0,08db4255286f,0,2018-11-05 10:00:00,-30.845301,0.0447,1,2018,11,5,10
1,08db4255286f,1,2018-11-05 10:00:05,-34.181801,0.0443,1,2018,11,5,10
2,08db4255286f,2,2018-11-05 10:00:10,-33.877102,0.0483,1,2018,11,5,10
3,08db4255286f,3,2018-11-05 10:00:15,-34.282101,0.0680,1,2018,11,5,10
4,08db4255286f,4,2018-11-05 10:00:20,-34.385799,0.0768,1,2018,11,5,10


In [5]:
train_series_multi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16289820 entries, 0 to 16289819
Data columns (total 10 columns):
 #   Column     Dtype         
---  ------     -----         
 0   series_id  object        
 1   step       uint32        
 2   timestamp  datetime64[us]
 3   anglez     float32       
 4   enmo       float32       
 5   awake      int64         
 6   year       int32         
 7   month      uint32        
 8   day        uint32        
 9   hour       uint32        
dtypes: datetime64[us](1), float32(2), int32(1), int64(1), object(1), uint32(4)
memory usage: 807.8+ MB


In [6]:
# %%time
# # inspired by https://www.kaggle.com/code/renatoreggiani/reduce-memory-usage-zzzs-cmi
# # with tweaks determined by the selected polars loading strategy
# # tweaks inspired by https://github.com/softhints/Pandas-Tutorials/blob/master/column/3.check-dtype-column-columns-pandas-dataframe.ipynb
# from pandas.api.types import is_datetime64_ns_dtype
# def reduce_mem_usage(df):
#     """ iterate through all numeric columns of a dataframe and modify the data type
#         to reduce memory usage.        
#     """
#     start_mem = df.memory_usage().sum() / 1024**2
#     print(f'Memory usage of dataframe is {start_mem:.2f} MB')
    
#     for col in df.columns:
#         col_type = df[col].dtype

#         if col_type != object and 'datetime' not in str(col_type):
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)  
#             else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float16)

#     end_mem = df.memory_usage().sum() / 1024**2
#     print(f'Memory usage after optimization is: {end_mem:.2f} MB')
#     decrease = 100 * (start_mem - end_mem) / start_mem
#     print(f'Decreased by {decrease:.2f}%')                                                                                    
    
#     return df

# train_series_multi = reduce_mem_usage(train_series_multi)

In [7]:
# features, feature_cols = [pl.col('hour')], ['hour']

# for mins in [5, 30, 60*2, 60*8] :
    
#     for var in ['enmo', 'anglez'] :
        
#         features += [
#             pl.col(var).rolling_mean(12 * mins, center=True, min_periods=1).abs().alias(f'{var}_{mins}m_mean'),
#             pl.col(var).rolling_max(12 * mins, center=True, min_periods=1).abs().alias(f'{var}_{mins}m_max'),
#             pl.col(var).rolling_std(12 * mins, center=True, min_periods=1).abs().alias(f'{var}_{mins}m_std')
#         ]

#         feature_cols += [ 
#             f'{var}_{mins}m_mean', f'{var}_{mins}m_max', f'{var}_{mins}m_std'
#         ]

#         # Getting first variations
#         features += [
#             (pl.col(var).diff().abs().rolling_mean(12 * mins, center=True, min_periods=1)*10).abs().alias(f'{var}_1v_{mins}m_mean'),
#             (pl.col(var).diff().abs().rolling_max(12 * mins, center=True, min_periods=1)*10).abs().alias(f'{var}_1v_{mins}m_max'),
#             (pl.col(var).diff().abs().rolling_std(12 * mins, center=True, min_periods=1)*10).abs().alias(f'{var}_1v_{mins}m_std')
#         ]

#         feature_cols += [ 
#             f'{var}_1v_{mins}m_mean', f'{var}_1v_{mins}m_max', f'{var}_1v_{mins}m_std'
#         ]

# id_cols = ['series_id', 'step', 'timestamp']

# train_series_multi = pl.DataFrame(train_series_multi)

# train_series_multi= train_series_multi.with_columns(features).select(id_cols + feature_cols)

In [8]:
# train_series_multi

In [9]:
# signal_awake = dict(zip(range(1440), np.sin(np.linspace(0, np.pi, 1440) + 0.208 * np.pi) ** 24))
# signal_onset = dict(zip(range(1440), np.sin(np.linspace(0, np.pi, 1440) + 0.555 * np.pi) ** 24))

def diff_feature(df: pd.Series, col: str, window_size: int = 60):
    ret = df.groupby('series_id')[col].diff(periods=window_size)
    ret = ret.fillna(ret.loc[ret.first_valid_index()])
    gc.collect()
    return ret


def rolling_feature(df: pd.Series, col: str, agg: str, window_size: int = 60):
    ret = df.groupby('series_id')[col].rolling(window_size, center=True).agg(agg)
    ret = ret.fillna(method='bfill').fillna(method='ffill')
    gc.collect()
    return ret


def create_features(df_, window_size=60, verbose=False):
    start_time = time()
    df = df_.copy()
    
    if 'awake' in df.columns:
        df.awake = df.awake.astype(np.uint8)
    
    df['weekday'] = df['timestamp'].dt.weekday.astype(np.int8)
#     df['signal_onset'] = (df.timestamp.dt.hour * 60 + df.timestamp.dt.minute).map(signal_onset).astype(np.float16)
#     df['signal_awake'] = (df.timestamp.dt.hour * 60 + df.timestamp.dt.minute).map(signal_awake).astype(np.float16)
    
    df['anglez'] = abs(df['anglez'])
    
    df['enmo_rolling_mean']     = rolling_feature(df, col='enmo', agg='mean', window_size=window_size).values
#     df['enmo_rolling_min']      = rolling_feature(df, col='enmo', agg='min', window_size=window_size).astype('float16').values
    df['enmo_rolling_max']      = rolling_feature(df, col='enmo', agg='max', window_size=window_size).values
    df['enmo_rolling_std']      = rolling_feature(df, col='enmo', agg='std', window_size=window_size).astype('float32').values
    df['enmo_rolling_median']   = rolling_feature(df, col='enmo', agg='median', window_size=window_size).values
    
    df['anglez_rolling_mean']     = rolling_feature(df, col='anglez', agg='mean', window_size=window_size).values
#     df['anglez_rolling_min']      = rolling_feature(df, col='anglez', agg='min', window_size=window_size).astype('float16').values
    df['anglez_rolling_max']      = rolling_feature(df, col='anglez', agg='max', window_size=window_size).values
    df['anglez_rolling_std']      = rolling_feature(df, col='anglez', agg='std', window_size=window_size).astype('float32').values
    df['anglez_rolling_median']   = rolling_feature(df, col='anglez', agg='median', window_size=window_size).values

    df['enmo_prev_diff_rolling_mean']  = diff_feature(df, col='enmo_rolling_mean', window_size=window_size).astype('float32').values
    df['enmo_prev_diff_rolling_max']   = diff_feature(df, col='enmo_rolling_max', window_size=window_size).astype('float32').values
    
    df['anglez_prev_diff_rolling_mean']  = diff_feature(df, col='anglez_rolling_mean', window_size=window_size).astype('float32').values
    df['anglez_prev_diff_rolling_max']   = diff_feature(df, col='anglez_rolling_max', window_size=window_size).astype('float32').values

    gc.collect()
    
    x = (df.anglez - df.anglez_rolling_median).abs().to_frame(name='ad').assign(series_id=df.series_id)
    x = x.groupby('series_id')['ad'].rolling(window_size, center=True).agg('median')
    df['anglez_mad'] = x.fillna(method='bfill').fillna(method='ffill').astype('float32').values
    
    x = (df.enmo - df.enmo_rolling_median).abs().to_frame(name='ad').assign(series_id=df.series_id)
    x = x.groupby('series_id')['ad'].rolling(window_size, center=True).agg('median')
    df['enmo_mad'] = x.fillna(method='bfill').fillna(method='ffill').astype('float32').values

    z = np.maximum(0., df.enmo - 0.02)
    z = z.to_frame(name='lids').assign(series_id=df.series_id)
    z = rolling_feature(z, col='lids', agg='sum', window_size=120)
    z = 100 / (z + 1)
    z = z.to_frame().reset_index()
    z = rolling_feature(z, col='lids', agg='mean', window_size=360)
    df['lids'] = z.astype('float32').values
    
    del (df['enmo_rolling_median'], df['anglez_rolling_median'])
    
    cols_astype_float32 = df.columns[df.columns.str.contains('enmo|anglez|signal|lids')].tolist()
    for col in cols_astype_float32:
        df[col] = df[col].astype('float32')
    
    gc.collect()    
    
    end_time = time()
    eta = int(end_time - start_time)
    if verbose:
        print(f'Done in: {eta}s')
    
    return df

In [10]:
window_size = 36
# 1 step = 5s, 10 steps = 50s, 20 steps = 100s
# 1 min = 12 steps, 3 mins = 36 steps, 5 mins = 60 steps

train = create_features(train_series_multi, window_size=window_size, verbose=True)
train.info()

Done in: 411s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16289820 entries, 0 to 16289819
Data columns (total 24 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   series_id                      object        
 1   step                           uint32        
 2   timestamp                      datetime64[us]
 3   anglez                         float32       
 4   enmo                           float32       
 5   awake                          uint8         
 6   year                           int32         
 7   month                          uint32        
 8   day                            uint32        
 9   hour                           uint32        
 10  weekday                        int8          
 11  enmo_rolling_mean              float32       
 12  enmo_rolling_max               float32       
 13  enmo_rolling_std               float32       
 14  anglez_rolling_mean            float32       
 15 

In [11]:
train

,series_id,step,timestamp,anglez,enmo,awake,year,month,day,hour,...,anglez_rolling_mean,anglez_rolling_max,anglez_rolling_std,enmo_prev_diff_rolling_mean,enmo_prev_diff_rolling_max,anglez_prev_diff_rolling_mean,anglez_prev_diff_rolling_max,anglez_mad,enmo_mad,lids
0,08db4255286f,0,2018-11-05 10:00:00,30.845301,0.0447,1,2018,11,5,10,...,28.502230,34.925598,4.160982,0.022592,0.0907,-1.000903,-1.678898,2.817699,0.008675,13.032943
1,08db4255286f,1,2018-11-05 10:00:05,34.181801,0.0443,1,2018,11,5,10,...,28.502230,34.925598,4.160982,0.022592,0.0907,-1.000903,-1.678898,2.817699,0.008675,13.032943
2,08db4255286f,2,2018-11-05 10:00:10,33.877102,0.0483,1,2018,11,5,10,...,28.502230,34.925598,4.160982,0.022592,0.0907,-1.000903,-1.678898,2.817699,0.008675,13.032943
3,08db4255286f,3,2018-11-05 10:00:15,34.282101,0.0680,1,2018,11,5,10,...,28.502230,34.925598,4.160982,0.022592,0.0907,-1.000903,-1.678898,2.817699,0.008675,13.032943
4,08db4255286f,4,2018-11-05 10:00:20,34.385799,0.0768,1,2018,11,5,10,...,28.502230,34.925598,4.160982,0.022592,0.0907,-1.000903,-1.678898,2.817699,0.008675,13.032943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16289815,390b487231ce,396175,2018-05-19 14:29:35,1.424500,0.0502,2,2018,5,19,14,...,14.911117,20.717199,3.446494,0.007842,0.0105,4.163167,2.180199,13.027249,0.082350,58.615932
16289816,390b487231ce,396176,2018-05-19 14:29:40,2.564200,0.0498,2,2018,5,19,14,...,14.911117,20.717199,3.446494,0.006892,0.0105,4.001864,2.180199,13.027249,0.082350,58.615932
16289817,390b487231ce,396177,2018-05-19 14:29:45,4.081600,0.0428,2,2018,5,19,14,...,14.911117,20.717199,3.446494,0.005825,0.0105,3.775675,2.180199,13.027249,0.082350,58.615932
16289818,390b487231ce,396178,2018-05-19 14:29:50,12.700600,0.0642,2,2018,5,19,14,...,14.911117,20.717199,3.446494,0.005044,0.0105,3.523586,2.180199,13.027249,0.082350,58.615932


In [12]:
train.to_parquet("feature_engineer_V1.parquet")

In [13]:
test = create_features(test_series, window_size=window_size, verbose=True)
test.info()

Done in: 1s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   series_id                      450 non-null    object        
 1   step                           450 non-null    uint32        
 2   timestamp                      450 non-null    datetime64[us]
 3   anglez                         450 non-null    float32       
 4   enmo                           450 non-null    float32       
 5   year                           450 non-null    int32         
 6   month                          450 non-null    uint32        
 7   day                            450 non-null    uint32        
 8   hour                           450 non-null    uint32        
 9   weekday                        450 non-null    int8          
 10  enmo_rolling_mean              450 non-null    float32       
 11  enmo_ro

In [14]:
test

,series_id,step,timestamp,anglez,enmo,year,month,day,hour,weekday,...,anglez_rolling_mean,anglez_rolling_max,anglez_rolling_std,enmo_prev_diff_rolling_mean,enmo_prev_diff_rolling_max,anglez_prev_diff_rolling_mean,anglez_prev_diff_rolling_max,anglez_mad,enmo_mad,lids
0,038441c925bb,0,2018-08-14 15:30:00,2.636700,0.0217,2018,8,14,15,1,...,30.612183,79.998703,33.031490,-0.004022,0.0000,38.700752,0.026596,2.267200,0.0003,NaN
1,038441c925bb,1,2018-08-14 15:30:05,2.636800,0.0215,2018,8,14,15,1,...,30.612183,79.998703,33.031490,-0.004022,0.0000,38.700752,0.026596,2.267200,0.0003,NaN
2,038441c925bb,2,2018-08-14 15:30:10,2.637000,0.0216,2018,8,14,15,1,...,30.612183,79.998703,33.031490,-0.004022,0.0000,38.700752,0.026596,2.267200,0.0003,NaN
3,038441c925bb,3,2018-08-14 15:30:15,2.636800,0.0213,2018,8,14,15,1,...,30.612183,79.998703,33.031490,-0.004022,0.0000,38.700752,0.026596,2.267200,0.0003,NaN
4,038441c925bb,4,2018-08-14 15:30:20,2.636800,0.0215,2018,8,14,15,1,...,30.612183,79.998703,33.031490,-0.004022,0.0000,38.700752,0.026596,2.267200,0.0003,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,0402a003dae9,145,2018-12-18 12:57:05,59.696899,0.0601,2018,12,18,12,1,...,44.198860,81.162102,22.042067,-0.027175,-0.5459,-2.458192,0.000000,16.406851,0.0112,NaN
446,0402a003dae9,146,2018-12-18 12:57:10,35.656601,0.0427,2018,12,18,12,1,...,44.198860,81.162102,22.042067,-0.021614,-0.5459,-3.731261,0.000000,16.406851,0.0112,NaN
447,0402a003dae9,147,2018-12-18 12:57:15,21.582399,0.0309,2018,12,18,12,1,...,44.198860,81.162102,22.042067,-0.016072,-0.5459,-4.788300,0.000000,16.406851,0.0112,NaN
448,0402a003dae9,148,2018-12-18 12:57:20,42.616001,0.0328,2018,12,18,12,1,...,44.198860,81.162102,22.042067,-0.013503,-0.5459,-5.285600,0.000000,16.406851,0.0112,NaN
